# Problema 1 sección 5.4 ejercicio 5

5. In Chapter 4, we used logistic regression to predict the probability of
default using income and balance on the Default data set. We will
now estimate the test error of this logistic regression model using the
validation set approach. Do not forget to set a random seed before
beginning your analysis.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ISLP import load_data
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
df=load_data("Default")
df["default"]=df["default"].map({"Yes":1,"No":0})
df["student"]=df["student"].map({"Yes":1,"No":0})
print(df.shape)
df.head()

(10000, 4)


,default,student,balance,income
0,0,0,729.526495,44361.625074
1,0,1,817.180407,12106.134700
2,0,0,1073.549164,31767.138947
3,0,0,529.250605,35704.493935
4,0,0,785.655883,38463.495879


(a) Fit a logistic regression model that uses `income` and `balance` to
predict `default`.

In [3]:
# Agregando constante para el intercepto
X = sm.add_constant(df[['balance', 'income']])
y = df['default']
X.head()

,const,balance,income
0,1.0,729.526495,44361.625074
1,1.0,817.180407,12106.134700
2,1.0,1073.549164,31767.138947
3,1.0,529.250605,35704.493935
4,1.0,785.655883,38463.495879


In [4]:
# Ajustando modelo de Regresión Logistica
model = sm.Logit(y, X).fit()
model.summary()

Optimization terminated successfully.
         Current function value: 0.078948
         Iterations 10


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                default   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9997
Method:                           MLE   Df Model:                            2
Date:                Tue, 21 Nov 2023   Pseudo R-squ.:                  0.4594
Time:                        18:18:13   Log-Likelihood:                -789.48
converged:                       True   LL-Null:                       -1460.3
Covariance Type:            nonrobust   LLR p-value:                4.541e-292
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -11.5405      0.435    -26.544      0.000     -12.393     -10.688
balance        0.0056      0.000     24.835      0.000       0.005       0.006
income      2.081e-05   4.99e-06      4.174      0.000     1.1e-05    3.06e-05
==============================================================================

Possibly complete quasi-separation: A fraction 0.14 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

(b) Using the validation set approach, estimate the test error of this
model. In order to do this, you must perform the following steps:

i. Split the sample set into a training set and a validation set.

ii. Fit a multiple logistic regression model using only the training observations.

iii. Obtain a prediction of default status for each individual in
the validation set by computing the posterior probability of
default for that individual, and classifying the individual to
the default category if the posterior probability is greater
than 0.5.

iv. Compute the validation set error, which is the fraction of
the observations in the validation set that are misclassifed

In [5]:
# i. Separar la data en training y validation set
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# ii. Ajustar el modelo usando el training set
model = sm.Logit(y_train, X_train).fit()

# iii. Obtener predicción de validation con un umbral de decisión del 0.5
y_val_pred_proba = model.predict(X_val)
y_val_pred = (y_val_pred_proba > 0.5).astype(int)

# iv. Caluclar el e rror en validation set
validation_error = (1 - accuracy_score(y_val, y_val_pred))*100

# Display the validation set error
display(model.summary())

print(f'Validation Set Accuracy: {accuracy_score(y_val, y_val_pred)*100:.2f}%')
print(f'Validation Set Error: {validation_error:.2f}%')

Optimization terminated successfully.
         Current function value: 0.076230
         Iterations 10


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                default   No. Observations:                 8000
Model:                          Logit   Df Residuals:                     7997
Method:                           MLE   Df Model:                            2
Date:                Tue, 21 Nov 2023   Pseudo R-squ.:                  0.4743
Time:                        18:18:13   Log-Likelihood:                -609.84
converged:                       True   LL-Null:                       -1160.2
Covariance Type:            nonrobust   LLR p-value:                9.963e-240
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -11.8297      0.506    -23.387      0.000     -12.821     -10.838
balance        0.0058      0.000     22.038      0.000       0.005       0.006
income      2.075e-05   5.64e-06      3.682      0.000    9.71e-06    3.18e-05
==============================================================================

Possibly complete quasi-separation: A fraction 0.16 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

Validation Set Accuracy: 96.95%
Validation Set Error: 3.05%


(c) Repeat the process in (b) three times, using three diferent splits
of the observations into a training set and a validation set. Comment on the results obtained.

In [6]:
random_seed=50

# Repetir el proceso 3 veces con diferentes particiones del conjunto de datos
for i in range(3):
    # i. Separar la data en training y validation set
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=random_seed + i)

    # ii. Ajustar el modelo usando el training set
    model = sm.Logit(y_train, X_train).fit()

    # iii. Obtener predicción de validation con un umbral de decisión del 0.5
    y_val_pred_proba = model.predict(X_val)
    y_val_pred = (y_val_pred_proba > 0.5).astype(int)

    # iv. Caluclar el error en validation set
    validation_error_percentage = (1 - accuracy_score(y_val, y_val_pred)) * 100

    # Mostrar los resultados de cada iretación
    print(f'\nIteration {i + 1}: Validation Set Error: {validation_error_percentage:.2f}%\n')

Optimization terminated successfully.
         Current function value: 0.077885
         Iterations 10

Iteration 1: Validation Set Error: 2.90%

Optimization terminated successfully.
         Current function value: 0.077232
         Iterations 10

Iteration 2: Validation Set Error: 3.00%

Optimization terminated successfully.
         Current function value: 0.078456
         Iterations 10

Iteration 3: Validation Set Error: 2.60%



Se puede evidenciar que dada la distirbución aleatoria del conjunto de training y validation, en cada repetición, se asignan observaciones diferentes a cada uno de estos conjuntos y, por lo tanto, se obtienen estimaciones del error diferentes.

(d) Now consider a logistic regression model that predicts the probability of default using income, balance, and a dummy variable
for student. Estimate the test error for this model using the validation set approach. Comment on whether or not including a
dummy variable for student leads to a reduction in the test error
rate.

In [7]:
# Agregando constante al conjuinto de Variables X
X = sm.add_constant(df[['balance', 'income', 'student']])
y = df['default']

# i. Separar la data en training y validation set
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# ii. Ajustar el modelo usando el training set
model = sm.Logit(y_train, X_train).fit()

# iii. Obtener predicción de validation con un umbral de decisión del 0.5
y_val_pred_proba = model.predict(X_val)
y_val_pred = (y_val_pred_proba > 0.5).astype(int)

# iv. Caluclar el error en validation set
validation_error_percentage = (1 - accuracy_score(y_val, y_val_pred)) * 100

# Mostrar el error sobre el conjunto de validación y los resultados
display(model.summary())
print(f'Validation Set Error: {validation_error_percentage:.2f}%')

Optimization terminated successfully.
         Current function value: 0.076032
         Iterations 10


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                default   No. Observations:                 8000
Model:                          Logit   Df Residuals:                     7996
Method:                           MLE   Df Model:                            3
Date:                Tue, 21 Nov 2023   Pseudo R-squ.:                  0.4757
Time:                        18:18:13   Log-Likelihood:                -608.26
converged:                       True   LL-Null:                       -1160.2
Covariance Type:            nonrobust   LLR p-value:                5.404e-239
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -11.3117      0.577    -19.614      0.000     -12.442     -10.181
balance        0.0059      0.000     21.933      0.000       0.005       0.006
income       7.21e-06   9.46e-06      0.763      0.446   -1.13e-05    2.57e-05
student       -0.4881      0.273     -1.790      0.074      -1.023       0.046
==============================================================================

Possibly complete quasi-separation: A fraction 0.17 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

Validation Set Error: 3.05%


En este caso, agregar la variable binaria `student` no tienen ningún efecto sobre la estimación del error sobre el conjunto de validación. Dato que da 3.05% (lo mismo que dió el modelo sin esta variable con el misms `random_state`)